In [2]:
import numpy as np
import pandas as pd

import nltk


In [3]:
train=pd.read_csv('spam.csv')

In [4]:
test= pd.read_csv('test.csv')
test.v2=test.v2.astype(str)

In [5]:
train= train.drop_duplicates(subset='v2', keep=False)
corpus= train['v2'].to_list()


In [6]:
def pre_process(sentence):
    
    stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    
    sentence = sentence.lower()
    
   
    sentence= sentence.split()
    sentence= [word for word in sentence if word not in stopwords]
    sentence=[ ele for ele in sentence if ele not in punc]
    sentence=(' ').join(sentence)
   
    return sentence

In [7]:
corpus=[pre_process(sentence) for sentence in corpus]

In [8]:
labels= np.array(train['v1'].to_list())

In [9]:
print(labels)

[0. 0. 0. ... 0. 0. 0.]


In [10]:
train_sentences, test_sentences=corpus[ 0: 7000], corpus[ 7000: :]

train_labels, test_labels= labels[ 0: 7000], labels[ 7000: :]

In [11]:
vocab_size = 20000
max_length=None
embedding_dim = 16
trunc_type='post'
padding= 'post'
oov_tok = "<OOV>"

In [12]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import tensorflow as tf

2022-07-01 01:09:05.281149: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-01 01:09:05.281188: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [13]:
tokenizer= Tokenizer(num_words=vocab_size,oov_token=oov_tok)

In [14]:
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index


In [15]:
train_sequences=tokenizer.texts_to_sequences(train_sentences)
train_padded= pad_sequences(train_sequences, dtype='int32',padding=padding,value=0.0)

test_sequences=tokenizer.texts_to_sequences(test_sentences)
# test_padded= pad_sequences(test_sequences, maxlen=max_length,dtype='int32',padding=padding,value=0.0)

In [16]:
lstm_dim = 200
dense_dim=20

In [17]:
model_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=train_padded.shape[1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_dim, dropout=0.2)),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

checkpoint=tf.keras.callbacks.ModelCheckpoint('keras_model',save_best_only=True, monitor='val_loss')
model_lstm.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model_lstm.summary()



2022-07-01 01:09:08.321940: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-01 01:09:08.321965: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-01 01:09:08.321982: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ice-82hu): /proc/driver/nvidia/version does not exist
2022-07-01 01:09:08.322475: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 104, 16)           320000    
                                                                 
 bidirectional (Bidirectiona  (None, 400)              347200    
 l)                                                              
                                                                 
 dense (Dense)               (None, 20)                8020      
                                                                 
 dense_1 (Dense)             (None, 1)                 21        
                                                                 
Total params: 675,241
Trainable params: 675,241
Non-trainable params: 0
_________________________________________________________________


In [20]:
NUM_EPOCHS = 3
BATCH_SIZE = 128


history_lstm = model_lstm.fit(train_padded, train_labels, batch_size=128, epochs=3, callbacks=[checkpoint])

Epoch 1/3
39/39 [==============================] - 13s 328ms/step - loss: 0.0474 - accuracy: 0.9867
Epoch 2/3
39/39 [==============================] - 13s 328ms/step - loss: 0.0224 - accuracy: 0.9941
Epoch 3/3
39/39 [==============================] - 13s 330ms/step - loss: 0.0158 - accuracy: 0.9955


In [21]:
from keras.models import load_model 
# model = load_model('Keras_Model')
test_sentences=test['v2'].to_list()
test_sentences=[pre_process(sentence) for sentence in test_sentences]

In [22]:
test_sequences=tokenizer.texts_to_sequences(test_sentences)
test_padded= pad_sequences(test_sequences, maxlen=max_length,dtype='int32',padding=padding,value=0.0)
test_padded.resize(3263,104)

In [23]:
prediction = model_lstm.predict(test_padded)

102/102 [==============================] - 7s 62ms/step


In [24]:
prediction

array([[6.8346661e-04],
       [3.6026622e-04],
       [9.9999958e-01],
       ...,
       [2.7197590e-03],
       [2.7197590e-03],
       [2.7197562e-03]], dtype=float32)

In [25]:
prediction[prediction>=0.5]=1
prediction[prediction<0.5]=0

In [26]:
sample_submission=pd.read_csv('sample_submission.csv')

In [27]:
sample_submission['FUCKAS']=prediction
sample_submission['FUCKAS']=sample_submission['FUCKAS'].astype(int)

In [28]:
sample_submission.to_csv('submission.csv', index=False)

In [29]:
sss=pd.read_csv('test.csv')

In [32]:
XXX=sss.iloc[:52,0]

In [33]:
XXX

0     0.0
1     0.0
2     1.0
3     0.0
4     0.0
5     1.0
6     0.0
7     0.0
8     1.0
9     1.0
10    0.0
11    1.0
12    1.0
13    0.0
14    0.0
15    1.0
16    0.0
17    0.0
18    0.0
19    1.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    0.0
31    0.0
32    0.0
33    0.0
34    1.0
35    0.0
36    0.0
37    0.0
38    0.0
39    0.0
40    0.0
41    0.0
42    1.0
43    0.0
44    0.0
45    0.0
46    0.0
47    0.0
48    0.0
49    0.0
50    0.0
51    0.0
Name: v1, dtype: float64

In [34]:
prediction

array([[0.],
       [0.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [37]:
from sklearn.metrics import confusion_matrix, accuracy_score
prediction2= prediction.resize(51,1)
# cm = confusion_matrix(XXX, prediction)
# print(cm)
# accuracy_score(XXX, prediction)

ValueError: cannot resize an array that references or is referenced
by another array in this way.
Use the np.resize function or refcheck=False